In [1]:
import datetime, json
import pandas as pd 
import geopy
import geopy.distance
import json


In [2]:
df = pd.read_csv('../../italy-dtp-2020-15.csv', header = 0, names=['days', 'time', 'lat', 'lon', 'discharge', 'temp', 'prec'])


In [3]:
df_drop = df.drop(columns = ['days', 'time', 'discharge', 'temp', 'prec'], inplace=False)
df_drop['coords'] = df_drop.apply(tuple, axis=1)



In [4]:
df_drop

,lat,lon,coords
0,44.750378,7.560520,"(44.750378, 7.5605197)"
1,44.750378,7.560520,"(44.750378, 7.5605197)"
2,44.750378,7.560520,"(44.750378, 7.5605197)"
3,44.750378,7.560520,"(44.750378, 7.5605197)"
4,44.750378,7.560520,"(44.750378, 7.5605197)"
...,...,...,...
3259957,41.234250,13.701608,"(41.23425, 13.701608)"
3259958,41.234250,13.701608,"(41.23425, 13.701608)"
3259959,41.234250,13.701608,"(41.23425, 13.701608)"
3259960,41.234250,13.701608,"(41.23425, 13.701608)"


In [5]:
ls = df_drop['coords'].unique()
ls

array([(44.750378, 7.5605197), (44.705326, 7.5624723),
       (44.66027, 7.564421), ..., (41.324547, 13.706987),
       (41.2794, 13.704295), (41.23425, 13.701608)], dtype=object)

In [6]:
with open('interesting.geojson', 'r') as fd:
    interest = json.load(fd)

jsonObj = {
    "type": "FeatureCollection",
    "features": []
}
i = 0
for l in ls:
    ptCoord = geopy.Point(l[0], l[1])
    for key in interest.keys():
        coordList = interest[key]['geoms']['coordinates']
        for coord in coordList:
            ptRiver = geopy.Point(coord[0], coord[1])
            dist = geopy.distance.distance(ptRiver, ptCoord).km
            if dist <= 2.6 :
                featureObj = {
                    "type": "Feature",
                    "name": key,
                    "id": i,
                    "geometry":{
                        "type":"Point",
                        "coordinates": [l[1], l[0]]
                    }
                }
                i += 1
                jsonObj["features"].append(featureObj)
                break
with open ('points_new.geojson', 'w') as fout:
    json.dump(jsonObj, fout)